In [3]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory


load_dotenv()

os.environ["ANTHROPIC_API_KEY"] = os.environ["ANTHROPIC_API_KEY"]
os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"]
os.environ["GOOGLE_API_KEY"] = os.environ["GOOGLE_API_KEY"]


In [4]:
models = {
    "gpt-3.5-turbo": ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    "claude-3-sonnet-20240229": ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0),
    "gemini-pro": ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
}
# Initialize conversation memory
memory = ConversationBufferMemory(return_messages=True)

In [5]:
# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Respond concisely."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create the chatbot chain
def create_chain(model_name):
    return (
        RunnablePassthrough.assign(history=memory.load_memory_variables) | {
            "response": prompt | models[model_name] | StrOutputParser()
        }
    ) | (lambda x: x["response"])

# Function to run the conversation
def chat(user_input, model_name="anthropic"):
    chain = create_chain(model_name)
    response = chain.invoke({"input": user_input})
    memory.save_context({"input": user_input}, {"output": response})
    return response

In [6]:
# Example usage
print(chat("Hello, how are you?"))
print(chat("What's the capital of France?"))
print(chat("Now, let's switch to OpenAI. Can you tell me about the Eiffel Tower?", "openai"))
print(chat("Finally, let's use Gemini. What's the population of Tokyo?", "gemini"))


KeyError: 'anthropic'

In [ ]:
# 5. User Interface

# In Jupyter Notebook, you can use widgets for a simple UI.
from ipywidgets import Text, Button, VBox
from IPython.display import display

def update_ui(user_input, model_name="anthropic"):
    response = chat(user_input, model_name)
    display(Text(value=f"You: {user_input}"))
    display(Text(value=f"AI: {response}"))

# Create input and button widgets
user_input = Text(description="Enter your message:")
send_button = Button(description="Send")

# Define behavior on button click
send_button.on_click(lambda _: update_ui(user_input.value))

# Display the UI
display(VBox([user_input, send_button]))

